In [16]:
import pandas as pd
import uuid

In [17]:
!pip install openpyxl


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
#Crear dimensiones base para variable a nivel sujeto
temaInteres = pd.DataFrame(columns=['id','nombre','descripcion'])

etapa= pd.DataFrame(columns=['id','numEtapa','nombreAnalisis','nombreBD','descripcion',
                             'eventoInicial','eventoFinal'])

fecha=pd.DataFrame(columns=['dia','mes','anio'])

evento=pd.DataFrame(columns=['id','nombre','descripcion'])

variable=pd.DataFrame(columns=['id','nombreBD','nombreAnalisis','unidades','edadCorregida','derivada',
                               'impacto','tipoDato','tipoVariable','nivelMedicion','longitudinal',
                               'descripcionCorta','descripcionLarga'])


In [19]:
# Conseguir dataframes de diccionario
file_dict=pd.ExcelFile('data/KMC-diccionario.xlsx')
df_etapas=pd.read_excel(file_dict, sheet_name='ETAPAS')
df_temas=pd.read_excel(file_dict, sheet_name='Temas de Interés')
df_variables=pd.read_excel(file_dict, sheet_name='KMC-12mEC-DIC-VAR-version2024')

In [20]:
#Poblar dimensión de eventos
eventos = pd.unique(
    pd.concat([
        df_etapas['Evento inicial (id-var)'],
        df_etapas['Evento-final (id-var)']
    ])
).tolist()

ids_eventos = [str(uuid.uuid4()) for _ in range(len(eventos))]

for i in range(len(eventos)):
    evento.loc[len(evento)] = [
        ids_eventos[i],
        eventos[i],
        ""
    ]

In [21]:
#Poblar dimensión de etapas
def encontrar_evento(nombre):
    fila=evento[evento['nombre']==nombre]
    return fila['id'].values[0]

ids_etapas = [str(uuid.uuid4()) for _ in range(len(df_etapas))]

for index, row in df_etapas.iterrows():
    etapa.loc[row['NUMERO ETAPA']]=[
        ids_etapas[index],
        row['NUMERO ETAPA'],
        row['ID-ETAPA'].lower(),
        row['NOMBRE CORTO'].lower(),
        row['DESCRIPCION'].lower(),
        encontrar_evento(row['Evento inicial (id-var)']),
        encontrar_evento(row['Evento-final (id-var)'])
    ]

In [22]:
#Poblar dimensión tema de interes
ids_temas = [str(uuid.uuid4()) for _ in range(len(df_temas))]

for index,row in df_temas.iterrows():
    temaInteres.loc[row['Número TdI']]=[
        ids_temas[index],
        row['ID-TdI'],
        row['DescripciOn']
    ]

In [23]:
#Poblar dimension fecha
fecha.loc[0]=[
    1,
    1,
    2025
]

In [24]:
#Poblar dimension variable

ids_vars = [str(uuid.uuid4()) for _ in range(len(df_variables))]

for index,row in df_variables.iterrows():
    variable.loc[index]=[
        ids_vars[index],
        row['NOMBRE EN LA BdeD'],
        row['VAR-ID'],
        row['UNITS'],
        False, #TODO: Edad coregida:De donde se saca esto?
        False, #TODO: Derivada: De donde se saca esto?
        False, #TODO: Impacto: ¿?
        row['VAR-TYPE'],
        False, #Tipo Variable
        False, #Nivel medicion
        False, #Longitudinal
        row['VAR-SHORT DESCRIPTION'],
        row['VAR-LONG DESCRIPTION']
    ]

In [25]:
#Crear diccionario de valor categorias
cols = [
    "VAL1", "SHORT-NAME1", "VAL2", "SHORT-NAME2",
    "VAL3", "SHORT-NAME3", "VAL4", "SHORT-NAME4", "VAL5", "SHORT-NAME5",
    "VAL6", "SHORT-NAME6", "VAL7", "SHORT-NAME7", "VAL8", "SHORT-NAME8",
    "VAL9", "SHORT-NAME9", "VAL10", "SHORT-NAME10", "VAL11", "SHORT-NAME11"
]

df_categorias=df_variables[cols]

unique_dicts = set()

def add_unique_dict(d):
    frozen = frozenset(d.items())
    if frozen not in unique_dicts:
        unique_dicts.add(frozen)

for index, row in df_categorias.iterrows():
    for i, col in  enumerate(df_categorias.columns):
        
        if i + 1 < len(df_categorias.columns):  
            siguiente_col = df_categorias.columns[i + 1]
        if pd.notna(row[col]) and 'VAL' in col and row[col] != ' ':
            dicc={
                'value':int(row[col]),
                'description':str(row[siguiente_col]).strip()
            }
            add_unique_dict(dicc)


In [26]:
valorCategoria=pd.DataFrame(columns=['id','codificacion','descripcion'])
for d in unique_dicts:
    data = dict(d) 
    nueva_fila = {
        'id': str(uuid.uuid4()),               
        'codificacion': data['value'],         
        'descripcion': data['description']     
    }
    valorCategoria.loc[len(valorCategoria)] = nueva_fila

valorCategoria

,id,codificacion,descripcion
0,49bda84c-096f-4946-87b7-a4c9038bed95,1,1 A 4
1,98ee3f36-c84f-4497-bf2f-542ce16dff15,4,+Salida-40semEPC
2,fe2fc507-df4f-41c9-8b38-418c9f0141ad,1,LME
3,36f6afe3-8e04-452a-ae3f-1c24878a9df2,2,no deserto por ahora
4,21db1f9b-03d4-484b-8422-9d47e65b64dc,2,Control por ARO MAS DE 50%
...,...,...,...
311,f13b53d0-61c4-4a01-946f-0853f51a96f0,14,Astigmatismo Hipermetrópico
312,b41ee518-f4bc-4b18-92d0-59c309405cd8,1,SI
313,861e63d5-1da9-4632-bd18-2d65230a40b1,1,Displasia Broncopulmonar
314,937b4866-751e-4a52-bf87-96e04f2e67d8,1,1 a 3


In [27]:
#Creacion tabla de hechos variable sujeto 
hechoEvaluarVariableSujero= pd.DataFrame(columns=['idVariable','idEtapa','idGrupoValorCategoria',
                                                  'idGrupoTemaInteres','idFechaModificacion','idFechaRegistro',
                                                  'idGrupoOperacion','idGrupoVariableLongitudinal'])

In [28]:
variable

,id,nombreBD,nombreAnalisis,unidades,edadCorregida,derivada,impacto,tipoDato,tipoVariable,nivelMedicion,longitudinal,descripcionCorta,descripcionLarga
0,15ede256-89a9-4606-9588-ef19ea48bcdd,CP_Prematuro,CP_Prematuro,NaN,False,False,False,INTEGER,False,False,False,Numero de prematuros,Numero de prematuros
1,8fcf32ee-895b-497b-b8bf-b9510d51fda9,Iden_Codigo,IDEN_Codigo,NaN,False,False,False,INTEGER,False,False,False,IdentificadorBebe,NaN
2,0d543ff3-a4b7-481d-8583-9c58b32cea5a,Iden_embarazoMultiple,CP_EmbarazoMultiple,NaN,False,False,False,BOOLEANO,False,False,False,si fue un embarazo multiple,NaN
3,a3d6b951-8111-4a49-813d-6e04907c1b45,Iden_FechaParto,PA_FechaParto,NaN,False,False,False,DATE11,False,False,False,Fecha del parto,NaN
4,b72135b1-e4b8-429d-8345-20e7247b5063,Iden_Sede,IDEN_Sede,NaN,False,False,False,INTEGER,False,False,False,sede del PMC,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,d110d676-0d62-42e4-b21c-0d3e9e8cc471,NaN,NaN,NaN,False,False,False,NaN,False,False,False,NaN,NaN
636,4993cde9-7f25-4509-9b7f-305d4c958e4b,NaN,NaN,NaN,False,False,False,NaN,False,False,False,NaN,NaN
637,f6410ae9-6968-4e97-963a-e1ba87ea18db,NaN,NaN,NaN,False,False,False,NaN,False,False,False,NaN,NaN
638,4924435a-9ac2-46ec-ae78-cc0e22fe6999,NaN,NaN,NaN,False,False,False,NaN,False,False,False,NaN,NaN


In [29]:
#Poblar tabla de hechos 


Anotaciones:
- La meta de donde sale?
- Los eventos no relacionados a la variable pero no a una etapa donde estan?
- En este momento se registraron las operaciones que se hicieron sobre las variables?
- ¿Cómo identificar variables longitudinales?
- Aun no tengo la BD  de indicadores